In [1]:
import collections
import itertools  
import matplotlib.pyplot as plt
import networkx as nx
import os
import tellurium as te
import tesbml
import os, sys
cwd = os.getcwd()
print(cwd)

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [2]:
parent_dir = os.path.abspath(os.path.join(cwd, os.pardir))
print(parent_dir)
os.chdir(parent_dir)
print(os.getcwd())

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint


In [3]:
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule
from SBMLLint.common.reaction import Reaction
from SBMLLint.common.simple_sbml import SimpleSBML
from SBMLLint.games import print_model as pm
from SBMLLint.games.som import SOM

In [148]:
num = 17
format_num = format(num, '03d')
document = tesbml.readSBML(os.path.join(os.getcwd(), 'SBMLLint/games/data/curated_' + format_num + '.xml'))
model = document.getModel()
pm.print_model(model)

<Model Hoefnagel2002_PyruvateBranches "Hoefnagel2002_PyruvateBranches">
R1: ADP + NAD + halfglucose -> ATP + NADH + pyruvate;
R2: NADH + pyruvate -> NAD + lactate;
R3: NAD + pyruvate + CoA -> NADH + AcCoA;
R4: AcCoA + PO4 -> CoA + AcP;
R5: ADP + AcP -> ATP + Ac;
R6: NADH + AcCoA -> NAD + CoA + AcO;
R7: NADH + AcO -> NAD + EtOH;
R8: pyruvate -> AcLac;
R9: AcLac -> AcetoinIn;
R10: AcetoinIn -> AcetoinOut;
R11: NADH + AcetoinIn -> NAD + Butanediol;
R12: ATP -> ADP;
R13: NADH + O2 -> NAD;
R14: AcLac -> AcetoinIn;


In [149]:
simple = SimpleSBML(model)

In [150]:
print(simple._getReactions())
len(simple._getReactions())

[<Reaction R1>, <Reaction R2>, <Reaction R3>, <Reaction R4>, <Reaction R5>, <Reaction R6>, <Reaction R7>, <Reaction R8>, <Reaction R9>, <Reaction R10>, <Reaction R11>, <Reaction R12>, <Reaction R13>, <Reaction R14>]


14

In [151]:
Reaction.reactions = []
Reaction.initialize(simple)
print(Reaction.reactions)
len(Reaction.reactions)
# makes sense because R9 and R14 are same;

[R1: ADP + NAD + halfglucose -> ATP + NADH + pyruvate; 2 * V_1 * (halfglucose / (2 * Kglc_1)) * (NAD / Knad_1) * (ADP / Kadp_1) / ((1 + halfglucose / (2 * Kglc_1) + pyruvate / Kpyr_1) * (1 + NAD / Knad_1 + NADH / Knadh_1) * (1 + ADP / Kadp_1 + ATP / Katp_1)), R2: NADH + pyruvate -> NAD + lactate; V_2 * ((pyruvate * NADH - lactate * NAD / Keq_2) / (Kpyr_2 * Knadh_2)) / ((1 + pyruvate / Kpyr_2 + lactate / Klac_2) * (1 + NADH / Knadh_2 + NAD / Knad_2)), R3: NAD + pyruvate + CoA -> NADH + AcCoA; V_3 * (pyruvate / Kpyr_3) * (NAD / Knad_3) * (CoA / Kcoa_3) * (NAD / (NAD + Ki_3 * NADH)) / ((1 + pyruvate / Kpyr_3) * (1 + NAD / Knad_3 + NADH / Knadh_3) * (1 + CoA / Kcoa_3 + AcCoA / Kaccoa_3)), R4: AcCoA + PO4 -> CoA + AcP; V_4 * ((AcCoA * PO4 - AcP * CoA / Keq_4) / (Kiaccoa_4 * Kpi_4)) / (1 + AcCoA / Kiaccoa_4 + PO4 / Kipi_4 + AcP / Kiacp_4 + CoA / Kicoa_4 + AcCoA * PO4 / (Kiaccoa_4 * Kpi_4) + AcP * CoA / (Kacp_4 * Kicoa_4)), R5: ADP + AcP -> ATP + Ac; V_5 * ((AcP * ADP - Ac * ATP / Keq_5) / (K

14

In [152]:
Reaction.reactions

[R1: ADP + NAD + halfglucose -> ATP + NADH + pyruvate; 2 * V_1 * (halfglucose / (2 * Kglc_1)) * (NAD / Knad_1) * (ADP / Kadp_1) / ((1 + halfglucose / (2 * Kglc_1) + pyruvate / Kpyr_1) * (1 + NAD / Knad_1 + NADH / Knadh_1) * (1 + ADP / Kadp_1 + ATP / Katp_1)),
 R2: NADH + pyruvate -> NAD + lactate; V_2 * ((pyruvate * NADH - lactate * NAD / Keq_2) / (Kpyr_2 * Knadh_2)) / ((1 + pyruvate / Kpyr_2 + lactate / Klac_2) * (1 + NADH / Knadh_2 + NAD / Knad_2)),
 R3: NAD + pyruvate + CoA -> NADH + AcCoA; V_3 * (pyruvate / Kpyr_3) * (NAD / Knad_3) * (CoA / Kcoa_3) * (NAD / (NAD + Ki_3 * NADH)) / ((1 + pyruvate / Kpyr_3) * (1 + NAD / Knad_3 + NADH / Knadh_3) * (1 + CoA / Kcoa_3 + AcCoA / Kaccoa_3)),
 R4: AcCoA + PO4 -> CoA + AcP; V_4 * ((AcCoA * PO4 - AcP * CoA / Keq_4) / (Kiaccoa_4 * Kpi_4)) / (1 + AcCoA / Kiaccoa_4 + PO4 / Kipi_4 + AcP / Kiacp_4 + CoA / Kicoa_4 + AcCoA * PO4 / (Kiaccoa_4 * Kpi_4) + AcP * CoA / (Kacp_4 * Kicoa_4)),
 R5: ADP + AcP -> ATP + Ac; V_5 * ((AcP * ADP - Ac * ATP / Keq_5) 

In [154]:
print(Molecule.molecules)

[ADP, NAD, halfglucose, ATP, NADH, pyruvate, lactate, CoA, AcCoA, PO4, AcP, Ac, AcO, EtOH, AcLac, AcetoinIn, AcetoinOut, Butanediol, O2]


In [155]:
SOM.initialize(Molecule.molecules)
print(SOM.soms)

[{ADP}, {NAD}, {halfglucose}, {ATP}, {NADH}, {pyruvate}, {lactate}, {CoA}, {AcCoA}, {PO4}, {AcP}, {Ac}, {AcO}, {EtOH}, {AcLac}, {AcetoinIn}, {AcetoinOut}, {Butanediol}, {O2}]


In [156]:
adp = SOM.findSOM(Molecule.getMolecule('ADP'))
atp = SOM.findSOM(Molecule.getMolecule('ATP'))
print(adp)
print(atp)

{ADP}
{ATP}


In [157]:
a = nx.DiGraph()
a.add_nodes_from(SOM.soms)
a.nodes()
a.add_edge(adp, atp)
print(a.nodes)
print(a.edges)

[{ADP}, {NAD}, {halfglucose}, {ATP}, {NADH}, {pyruvate}, {lactate}, {CoA}, {AcCoA}, {PO4}, {AcP}, {Ac}, {AcO}, {EtOH}, {AcLac}, {AcetoinIn}, {AcetoinOut}, {Butanediol}, {O2}]
[({ADP}, {ATP})]


In [158]:
class Arc(object):
    
    def __init__(self, source, destination, reaction):
        """
        :param SOM source:
        :param SOM destination:
        :param Reaction reaction:
        """
        self.source = source
        self.destination = destination
        self.reaction = reaction

In [184]:
SOM.initialize(Molecule.molecules)
print(SOM.soms)

[{ADP}, {NAD}, {halfglucose}, {ATP}, {NADH}, {pyruvate}, {lactate}, {CoA}, {AcCoA}, {PO4}, {AcP}, {Ac}, {AcO}, {EtOH}, {AcLac}, {AcetoinIn}, {AcetoinOut}, {Butanediol}, {O2}]


In [185]:
#Reaction.reactions[1].reactants[0].molecule

In [186]:
# maybe we can have multiple MESGraphs.. for each subgraph is a connected subgraph
class MESGraph(nx.DiGraph):
#     all = []

    def __init__(self, soms):
        """
        :param list-SOM soms:
        """
        super(MESGraph, self).__init__()
        self.add_nodes_from(soms)
        #self.arcs = set()
        self.identifier = self.makeId()
#         self.__class__.all.append(self)

    def __repr__(self):
        return self.identifier
    
    def makeId(self):
        identifier = ";"
        for som in list(self.nodes):
            identifier = identifier + som.identifier + "+"
        return identifier
    
    def processUniUniReaction(self, reaction):
        """
        :param Reaction reactions:
        """
        if reaction.category != cn.REACTION_1_1:
            pass
        else:
            reactant_som = SOM.findSOM(reaction.reactants[0].molecule)
            product_som = SOM.findSOM(reaction.products[0].molecule)
            if reactant_som == product_som:
                pass
            else:
                new_som = SOM.merge(reaction)
                self.remove_node(reactant_som)
                self.remove_node(product_som)
                self.add_node(new_som)
                return new_som
    
    def processUniMultiReaction(self, reaction):
        """
        :param Reaction reaction:
        """
        if (reaction.category != cn.REACTION_1_n):
            pass
        else:
            source = [SOM.findSOM(reaction.reactants[0].molecule)]
            destination = [SOM.findSOM(product.molecule) for product in reaction.products]
            print("reactant is: ", source)
            print("products is: ", destination)
            self.add_edges_from(itertools.product(source, destination), reaction=reaction)

    def processMultiUniReaction(self, reaction):
        """
        :param Reaction reaction:
        """
        if (reaction.category != cn.REACTION_n_1):
            pass
        else:
            source = [SOM.findSOM(reaction.products[0].molecule)]
            destination = [SOM.findSOM(reactant.molecule) for reactant in reaction.reactants]
            print("reactant is: ", source)
            print("products is: ", destination)
            self.add_edges_from(itertools.product(source, destination), reaction=reaction)
            
    @classmethod
    def findGraph(cls, molecule):
        """
        :param Molecule molecule:
        :return MESGraph/False:
        """
        som = SOM.findSOM(molecule)
        for graph in cls.all:
            for node in graph.nodes:
                if node == som:
                    return graph
        return False
        
     
    def analyze(self, reactions):
        """
        :param list-Reaction reactions:
        """
        uniuni = []
        unimulti = []
        multiuni = []
        multimulti = []
        for reaction in reactions:
            if reaction.category == cn.REACTION_1_1:
                uniuni.append(reaction)
            elif reaction.category == cn.REACTION_1_n:
                unimulti.append(reaction)
            elif reaction.category == cn.REACTION_n_1:
                multiuni.append(reaction)
            elif reaction.category == cn.REACTION_n_n:
                multimulti.append(reaction)
            
        for reaction in uniuni:
            self.processUniUniReaction(reaction)
        for reaction in unimulti:
            self.processUniMultiReaction(reaction)
        for reaction in multiuni:
            self.processMultiUniReaction(reaction)
        
        return 
        
#     @classmethod
#     def initialize(cls, soms):
#         """
#         :param list-SOM soms:
#         """
#         cls.all = []
#         for som in soms:
#             MESGraph(som)


In [187]:
m = MESGraph(SOM.soms)
m.nodes

NodeView(({ADP}, {NAD}, {halfglucose}, {ATP}, {NADH}, {pyruvate}, {lactate}, {CoA}, {AcCoA}, {PO4}, {AcP}, {Ac}, {AcO}, {EtOH}, {AcLac}, {AcetoinIn}, {AcetoinOut}, {Butanediol}, {O2}))

In [188]:
m.analyze(Reaction.reactions)
m.nodes

reactant is:  [{NAD}]
products is:  [{NADH}, {O2}]


NodeView(({NAD}, {halfglucose}, {NADH}, {lactate}, {CoA}, {AcCoA}, {PO4}, {AcP}, {Ac}, {AcO}, {EtOH}, {Butanediol}, {O2}, {AcLac, AcetoinIn, AcetoinOut, pyruvate}, {ADP, ATP}))

In [189]:
m.edges

OutEdgeView([({NAD}, {NADH}), ({NAD}, {O2})])

In [133]:
# reaction1 is 1-1 reaction
reaction1 = Reaction.reactions[7]
reaction1
reaction2 = Reaction.reactions[0]
reactants = [SOM.findSOM(reactant.molecule) for reactant in reaction2.reactants]
print(reactants)

[{ADP, ATP}, {NAD}, {halfglucose}]


In [137]:
for reaction in Reaction.reactions:
    print(reaction.category)

reaction_n_n
reaction_n_n
reaction_n_n
reaction_n_n
reaction_n_n
reaction_n_n
reaction_n_n
reaction_1_1
reaction_1_1
reaction_1_1
reaction_n_n
reaction_1_1
reaction_n_1
reaction_1_1
